#Setup

In [ ]:
!pip install apache-beam

In [ ]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions, StandardOptions
import argparse

parser = argparse.ArgumentParser()

parser.add_argument('--input',
                      dest='input',
                      required=True,
                      help='Input file to process.')
parser.add_argument('--output',
                      dest='output',
                      required=True,
                      help='Output file to write results to.')

path_args, pipeline_args = parser.parse_known_args()

inputs_pattern = path_args.input
outputs_prefix = path_args.output


options = PipelineOptions(pipeline_args)
p = beam.Pipeline(options=options)

attendance_count =   (
                          p
                          | 'Read lines' >> beam.io.ReadFromText(inputs_pattern)
                          | 'Split row' >> beam.Map(lambda record: record.split(','))
                          | 'Get all Accounts Dept Persons' >> beam.Filter(lambda record: record[3] == 'Accounts')
                          | 'Pair each employee with 1' >> beam.Map(lambda record: (record[1], 1))
                          | 'Group and sum' >> beam.CombinePerKey(sum)
                          | 'Format results' >> beam.Map(lambda employee_count: str(employee_count))
                          | 'Write results' >> beam.io.WriteToText(outputs_prefix)
                      )

p.run()

In [ ]:
import apache_beam as beam  # Importujemy Apache Beam, bibliotekę do przetwarzania danych w potokach.
from apache_beam.options.pipeline_options import PipelineOptions, StandardOptions  # Importujemy opcje konfiguracyjne dla potoku.
import argparse  # Importujemy argparse do obsługi argumentów wiersza poleceń.

# Tworzymy parser argumentów CLI, który pozwoli przekazać parametry wejściowe i wyjściowe.
parser = argparse.ArgumentParser()

# Dodajemy argument '--input', który określa ścieżkę do pliku wejściowego.
parser.add_argument('--input',
                      dest='input',       # Nazwa argumentu, do której przypiszemy wartość.
                      required=True,      # Argument jest wymagany - jeśli go nie podamy, skrypt się nie uruchomi.
                      help='Input file to process.')  # Opis wyświetlany w pomocy (-h).

# Dodajemy argument '--output', który określa ścieżkę do pliku wyjściowego.
parser.add_argument('--output',
                      dest='output',      # Nazwa argumentu.
                      required=True,      # Argument jest wymagany.
                      help='Output file to write results to.')  # Opis argumentu.

# Parsujemy argumenty CLI. `path_args` zawiera nasze argumenty wejścia i wyjścia,
# a `pipeline_args` przechowuje dodatkowe opcje konfiguracyjne dla Beam.
path_args, pipeline_args = parser.parse_known_args()

# Przypisujemy wartości argumentów CLI do zmiennych.
inputs_pattern = path_args.input    # Pobieramy wartość argumentu `--input` (ścieżka wejściowa).
outputs_prefix = path_args.output   # Pobieramy wartość argumentu `--output` (ścieżka wyjściowa).

# Tworzymy obiekt PipelineOptions, który przechowuje konfigurację potoku (np. runner, temp location).
options = PipelineOptions(pipeline_args)

# Tworzymy obiekt potoku Beam i przekazujemy mu opcje konfiguracji.
p = beam.Pipeline(options=options)

# Definiujemy potok przetwarzania danych.
attendance_count =   (
                          p
                          | 'Read lines' >> beam.io.ReadFromText(inputs_pattern)
                          # Odczytujemy linie z pliku wejściowego.
                          | 'Split row' >> beam.Map(lambda record: record.split(','))
                          # Dzielimy każdą linię na listę wartości po przecinku.
                          | 'Get all Accounts Dept Persons' >> beam.Filter(lambda record: record[3] == 'Accounts')
                          # Filtrujemy osoby z działu "Accounts".
                          | 'Pair each employee with 1' >> beam.Map(lambda record: (record[1], 1))
                          # Tworzymy pary (nazwa_pracownika, 1).
                          | 'Group and sum' >> beam.CombinePerKey(sum)
                          # Grupujemy według nazwiska i sumujemy wartości (liczymy wystąpienia pracowników).
                          | 'Format results' >> beam.Map(lambda employee_count: str(employee_count))
                          # Konwertujemy wynik na string do zapisania.
                          | 'Write results' >> beam.io.WriteToText(outputs_prefix)
                          # Zapisujemy wyniki do pliku wyjściowego.
                      )

# Uruchamiamy potok.
p.run()

In [ ]:
python beam_pipeline.py \
    --runner DataflowRunner \
    --project MOJ_PROJEKT \
    --region us-central1 \
    --input gs://$BUCKET_NAME/input.csv \
    --output gs://$BUCKET_NAME/output \
    --temp_location gs://$BUCKET_NAME/temp \
    --staging_location gs://$BUCKET_NAME/staging